In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os
import random
import matplotlib.pyplot as plt
import gc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
def top_k_accuracy_score(y_true, y_pred, k=1):
    ranks = np.sum(y_pred >= y_pred[y_true == 1].reshape(-1, 1), axis=1)
    return np.sum(ranks <= k) / ranks.shape[0]

In [ ]:
model_mn2 = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

In [ ]:
model_mn2.summary()

In [ ]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/flower_data/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/flower_data/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/flower_data/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]

X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
lda_per_filter = [
    LDA(n_components=5).fit(X_train[..., i], np.argmax(y_train, axis=1))
    for i in range(X_train.shape[-1])
]

In [ ]:
all_means = np.array([filter.xbar_ for filter in lda_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.scalings_[:, :5] for filter in lda_per_filter])
all_components = np.transpose(all_components, [1, 2, 0])

In [ ]:
filterwise_dot = np.einsum('ij...,jk...->...ik', X_train - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_train_lda = filterwise_dot

filterwise_dot = np.einsum('ij...,jk...->...ik', X_valid - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_valid_lda = filterwise_dot

filterwise_dot = np.einsum('ij...,jk...->...ik', X_test - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_test_lda = filterwise_dot

In [ ]:
idx = np.arange(X_train_lda.shape[0])
np.random.shuffle(idx)

X_train_lda_shuffled = X_train_lda[idx]
y_train_shuffled = y_train[idx]

X_train_lda_shuffled_flat = X_train_lda_shuffled.reshape(-1, 5 * 160)
X_valid_lda_flat = X_valid_lda.reshape(-1, 5 * 160)
X_test_aug_lda_flat = X_test_lda.reshape(-1, 5 * 160)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[5 * 160]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, kernel_initializer="lecun_normal", use_bias=False),
    keras.layers.ELU(),
    keras.layers.Dropout(rate=0.15),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(480, kernel_initializer="lecun_normal", use_bias=False),
    keras.layers.ELU(),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(102, activation="softmax")
])

model.summary()

model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

In [ ]:
model.fit(
    X_train_lda_shuffled_flat, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_lda_flat, y_valid),
    callbacks=[lr_scheduler]
)
gc.collect()

In [ ]:
model.save("model_lda_based")
!zip -r model_lda_basd.zip model_lda_based/

In [ ]:


# model = keras.models.load_model('model_mobilenetv2_regularized_oxford102')
print("Validation set accuracy")
preds_valid_1 = model(tf.reshape(X_valid, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_valid, preds_valid_1, k=1))
print("TOP 5", top_k_accuracy_score(y_valid, preds_valid_1, k=5))

print("Test set")
preds_test_aug_1 = model(tf.reshape(X_test, [-1, 7, 7, 160])).numpy()

print("TOP 1", top_k_accuracy_score(y_test, preds_test_aug_1, k=1))
print("TOP 5", top_k_accuracy_score(y_test, preds_test_aug_1, k=5))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[5, 160]),
    keras.layers.BatchNormalization(),
    keras.layers.SeparableConv1D(filters=320, kernel_size=5, depth_multiplier=3),
    keras.layers.ELU(),
    keras.layers.Flatten(),
    keras.layers.Dropout(rate=0.15),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(480, kernel_initializer="lecun_normal", use_bias=False),
    keras.layers.ELU(),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(102, activation="softmax")
])

model.summary()

model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

In [ ]:
model.fit(
    X_train_lda_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_lda, y_valid),
    callbacks=[lr_scheduler]
)
gc.collect()

In [ ]:
model.save("model_lda_sepconv")
!zip -r model_lda_sepconv.zip model_lda_sepconv/